# Homework 2: Word Similarity

Student Name:Mao Yundong

Student ID:882542

Python version used:2.7.13

## General info

<b>Due date</b>: 1pm, Sunday April 1st

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 5% of mark for class

<b>Overview</b>: In this homework, you'll be quantifying the similarity between pairs of words using the structure of WordNet and word co-occurrence in the Brown corpus, using PMI, LSA, and word2vec. You will quantify how well these methods work by comparing to a carefully filtered human annotated gold-standard.

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> It is recommended to use Python 2 but we accept Python 3 solutions</b>. Make sure you state which version you used in the beggining of this notebook.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Extra credit</b>: Each homework has a task which is optional with respect to getting full marks on the assignment, but that can be used to offset any points lost on this or any other homework assignment (but not the final project or the exam). We recommend you skip over this step on your first pass, and come back if you have time: the amount of effort required to receive full marks (1 point) on an extra credit question will be substantially more than earning the same amount of credit on other parts of the homework.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


<b>Instructions</b>: For this homework we will be comparing our methods against a popular dataset of word similarities called Similarity-353. You need to first obtain this data set, which can be downloaded <a href="http://www.cs.technion.ac.il/~gabr/resources/data/wordsim353/wordsim353.zip">here</a>. The file we will be using is called *combined.tab*. Except for the header (which should be stripped out), the file is tab formated with the first two columns corresponding to two words, and the third column representing a human-annotated similarity between the two words.

Assume the file *combined.tab* is located <b>in the same folder as this notebook</b>. You should load this file into a Python dictionary (NOTE: in Python, tuples of strings, i.e. ("tiger","cat") can serve as the keys of dictionaries). This dataset contains many rare words: we need to filter this dataset in order for it to be better suited to the resources we will use in this assignment. So your first goal is to filter this dataset to generate a smaller test set where you will evaluate your word similarity methods.

The first filtering is based on document frequencies in the Brown corpus, in order to remove rare words. In this assignment, we will be treating the <i>paragraphs</i> of the Brown corpus as our "documents", you can iterate over them by using the `paras` method of the corpus reader. You should start by creating a Python list where each element of the list is a set containing the word <b>types</b> from a different paragraph of the Brown corpus: the words should be lower-cased and lemmatized before they are added to the set (keep it around, because you will need this list again later on). Then, using the information in this corpus, calculate document frequencies and remove from your test set any word pairs where at least one of the two words has a document frequency of less than 10 in this corpus. 

The second filtering is based on words with highly ambiguous senses and involves using the NLTK interface to WordNet. Here, you should remove any words which do not have a *single primary sense*. We define single primary sense here as either having only one sense (i.e. only one synset), or where the count (as provided by the WordNet `count()` method for the lemmas associated with a synset) of the most common sense is at least five and at least five times larger than the next most common sense. Also, you should remove any words where the primary sense is not a noun (this information is also in the synset). Store the synset corresponding to this primary sense in a dictionary for use in the next section. Given this definition, remove any word pairs from the test set where at least one of the words does not contain a single primary sense or if the single primary sense is not a noun.

When you have applied these two filtering steps, print out all the pairs in your filtered test set (if you have done this correctly, the total should be more than 10, but less than 50).

(1.5 marks)

In [2]:
from nltk.corpus import brown
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
f = open("combined.tab","r")
strings = f.readlines()
testDic = {}
for i in range(1,len(strings)):
    temp = strings[i].split("\t")
    temp[2] = temp[2][0:-2]
    testDic[(temp[0],temp[1])] = float(temp[2])
# print result

def myLemmatize(lemmatizer,tWord):
    temp = lemmatizer.lemmatize(tWord)
    if len(temp) < len(tWord):
        return temp
    return lemmatizer.lemmatize(tWord,"v")
typeSetList = [set([myLemmatize(lemmatizer,temp.lower()) for l in ll for temp in l]) for ll in brown.paras()] 
# typeSetList = [set([lemmatizer.lemmatize(temp) for l in ll for temp in l]) for ll in brown.paras()]
# print len(typeSetList)
# print typeList[:10]


docFreq = {}
for typeSet in typeSetList:
    for word in typeSet:
        if word in docFreq:
            docFreq[word] +=1
        else:
            docFreq[word] = 1
            
for tup in testDic.keys():
    if tup[0] not in docFreq or tup[1] not in docFreq:
        del testDic[tup]
        continue
    if docFreq[tup[0]] < 10 or docFreq[tup[1]] < 10:
        del testDic[tup]

from nltk.corpus import wordnet
def getPrimarySynset(word):
    synsets = wordnet.synsets(word)
    if len(synsets) == 1:
        for lemma in synsets[0].lemmas():
            if lemma.name() == word:
                return synsets[0]
        return False
    elif len(synsets)>1:
        temp = sorted([[lemma.count(),synset] for synset in synsets for lemma in synset.lemmas() if lemma.name() == word],reverse=True)
        if len(temp) == 0:
            return False
        if len(temp) == 1:
            return temp[0][1]
        if temp[0][0] < 5 or temp[0][0] < temp[1][0]*5:
            return False
        else:
            return temp[0][1]
    else:
        return False
        
# result= []
testDicSyn = {}
for tup in testDic.keys():
    temp1 = getPrimarySynset(tup[0])
    temp2 = getPrimarySynset(tup[1])
    if temp1 is False or temp2 is False:
#         result.append(tup)
        del testDic[tup]
        continue
    if temp1.pos() != "n" or temp2.pos() != "n":
#         result.append(tup)
        del testDic[tup]
        continue
    testDicSyn[tup[0]] = temp1
    testDicSyn[tup[1]] = temp2

s =  set(testDic.keys())
print s


set([('psychology', 'science'), ('journey', 'car'), ('brother', 'monk'), ('equipment', 'maker'), ('coast', 'forest'), ('baby', 'mother'), ('canyon', 'landscape'), ('doctor', 'personnel'), ('soap', 'opera'), ('monk', 'slave'), ('psychology', 'doctor'), ('car', 'automobile'), ('type', 'kind'), ('psychology', 'health'), ('computer', 'laboratory'), ('century', 'year'), ('coast', 'hill'), ('psychology', 'mind'), ('luxury', 'car'), ('school', 'center'), ('journey', 'voyage'), ('stock', 'egg'), ('word', 'similarity'), ('professor', 'doctor'), ('planet', 'people'), ('hotel', 'reservation'), ('coast', 'shore')])


<b>Instructions</b>: Now you will create several dictionaries with similarity scores for pairs of words in your test set derived using the techniques discussed in class. The first of these is the Wu-Palmer scores derived from the hypernym relationships in WordNet, which you should calculate using the primary sense for each word derived above. You can use the built-in method included in the NLTK interface, you don't have to implement your own. When you're done,  print out the Python dictionary of word pair/similarity mappings. 

(0.5 marks)

In [41]:
from nltk.corpus import wordnet
wupDic = {}
for tup in s:
    wupDic[tup] = testDicSyn[tup[0]].wup_similarity(testDicSyn[tup[1]])
print wupDic


{('psychology', 'science'): 0.9411764705882353, ('journey', 'car'): 0.09523809523809523, ('brother', 'monk'): 0.5714285714285714, ('equipment', 'maker'): 0.5, ('coast', 'forest'): 0.16666666666666666, ('baby', 'mother'): 0.5, ('canyon', 'landscape'): 0.3333333333333333, ('doctor', 'personnel'): 0.13333333333333333, ('soap', 'opera'): 0.2222222222222222, ('monk', 'slave'): 0.6666666666666666, ('psychology', 'doctor'): 0.1111111111111111, ('car', 'automobile'): 1.0, ('type', 'kind'): 0.9473684210526315, ('psychology', 'health'): 0.21052631578947367, ('computer', 'laboratory'): 0.35294117647058826, ('century', 'year'): 0.8333333333333334, ('coast', 'hill'): 0.6666666666666666, ('psychology', 'mind'): 0.5714285714285714, ('luxury', 'car'): 0.1111111111111111, ('school', 'center'): 0.13333333333333333, ('journey', 'voyage'): 0.8571428571428571, ('stock', 'egg'): 0.11764705882352941, ('word', 'similarity'): 0.3333333333333333, ('professor', 'doctor'): 0.5, ('coast', 'shore'): 0.9090909090909

**Instructions:** Next, you will calculate Positive PMI (PPMI) for your word pairs using statistics derived from the Brown: you should use the same set up as you did to calculate document frequency above: paragraphs as documents, lemmatized, lower-cased, and with term frequency information removed by conversion to Python sets. You need to use the basic method for calculating PPMI introduced in class (and also in the reading) which is appropriate for any possible definition of co-occurrence (here, appearing in the same paragraph), but you should only calculate PPMI for the words in your test set. You must avoid building the entire co-occurrence matrix, instead you should keeping track of the sums you need for the probabilities as you go along. When you have calculated PMI for all the pairs, your code should print out the Python dictionary of word-pair/PPMI-similarity mappings.

(1 mark)

In [43]:
import math
total = len(typeSetList)
PMIMap = {}
for tup in testDic.keys():
    cxy = 0
    for typeSet in typeSetList:
        if tup[0] in typeSet and tup[1] in typeSet:
            cxy+=1
    cx,cy = docFreq[tup[0]],docFreq[tup[1]]
#     PMIMap[tup] = log2 cxy -log2 total - (log2 cx - log2total +log2 cy - log2total)
    if cxy == 0:
        PMIMap[tup] = 0
    else:
        PMIMap[tup] = math.log(cxy,2) -math.log(cx,2) - math.log(cy,2) + math.log(total,2)
print PMIMap


{('psychology', 'science'): 5.07849712711011, ('journey', 'car'): 0, ('brother', 'monk'): 2.8992677183777076, ('equipment', 'maker'): 4.283313403192922, ('coast', 'forest'): 2.9823361803398516, ('baby', 'mother'): 3.0992782344952765, ('canyon', 'landscape'): 0, ('doctor', 'personnel'): 2.1631267570095414, ('soap', 'opera'): 4.221195813265069, ('monk', 'slave'): 0, ('psychology', 'doctor'): 3.5070811582269013, ('car', 'automobile'): 3.2849280592550194, ('type', 'kind'): 0.6037196020058051, ('psychology', 'health'): 0, ('computer', 'laboratory'): 0, ('century', 'year'): 0.8552119329800814, ('coast', 'hill'): 1.1448891931258114, ('psychology', 'mind'): 2.7673960627071033, ('luxury', 'car'): 2.272328022475387, ('school', 'center'): 0.6272445293166591, ('journey', 'voyage'): 0, ('stock', 'egg'): 1.7655163294888805, ('word', 'similarity'): 0, ('professor', 'doctor'): 0, ('planet', 'people'): 0.40723143430848374, ('hotel', 'reservation'): 2.8910472115727384, ('coast', 'shore'): 4.562576270818

**Instructions:** Next, you will derive similarity scores using the LSA method, i.e. apply SVD and truncate to get a dense vector and then use cosine similarity between the two vectors for each word pair. You can use the Distributed Semantics notebook as a starting point, but note that since you are interested here in word semantics, you will be constructing a matrix where the (non-sparse) rows correspond to words in the vocabulary, and the (sparse) columns correspond to the texts where they appear (this is the opposite of the notebook). Again, use the Brown corpus, in the same format as with PMI and document frequency. After you have a matrix in the correct format, use truncatedSVD in Sci-kit learn to produce dense vectors of length 500, and then use cosine similarity to produce similarities for your word pairs. Print out the corresponding Python dictionary.

(1 mark)

In [44]:
from sklearn.metrics.pairwise import cosine_similarity
inm = {}
for i,c in enumerate(docFreq.keys()):
    inm[c] = i

from scipy.sparse import coo_matrix
row,column = [],[]
for i,ts in enumerate(typeSetList):
    for c in ts:  
        row.append(inm[c])
    column.extend([i]*len(ts))
data = [1]*len(row)
matr = coo_matrix((data,(row,column)),shape=(len(docFreq.keys()),len(typeSetList)))

    
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=500)
matr2= svd.fit_transform(matr) 
cosinM = {}
for tup in testDic.keys():
    X = [matr2[inm[tup[0]]]]
    Y = [matr2[inm[tup[1]]]]
    cosinM[tup] = cosine_similarity(X,Y)[0][0]
print cosinM

{('psychology', 'science'): 0.25615392405594856, ('journey', 'car'): 0.019177602934838653, ('brother', 'monk'): 0.055694743999354604, ('equipment', 'maker'): 0.257558891306331, ('coast', 'forest'): 0.13823166471832737, ('baby', 'mother'): 0.33248225678052673, ('canyon', 'landscape'): 0.10198870273699764, ('doctor', 'personnel'): 0.06664416764415962, ('soap', 'opera'): 0.04010046458166099, ('monk', 'slave'): 0.010899653322212521, ('psychology', 'doctor'): 0.09515460048604511, ('car', 'automobile'): 0.36338997998628, ('type', 'kind'): 0.024431189446792127, ('psychology', 'health'): 0.044020577589586045, ('computer', 'laboratory'): 0.13653776462715295, ('century', 'year'): 0.0720037095494849, ('coast', 'hill'): 0.23966213708484016, ('psychology', 'mind'): 0.11265680841505926, ('luxury', 'car'): 0.08765210552062344, ('school', 'center'): 0.042095642314992486, ('journey', 'voyage'): 0.13541990619545374, ('stock', 'egg'): 0.09317549370196715, ('word', 'similarity'): -0.005652176335695321, ('

**Instructions:** Next, you will derive a similarity score from word2vec vectors, using the Gensim interface. Check the Gensim word2vec tutorial for details on the API: https://radimrehurek.com/gensim/models/word2vec.html. Again, you should use the Brown for this, but for word2vec you don't need to worry about paragraphs: feel free to train your model at the sentence level instead. Your vectors should have the same number of dimensions as LSA (500), and you need to run for 50 iterations. This may take a while (several minutes), but that's okay, you won't be marked based on the speed of this. You should extract the similarites you need directly from the Gensim model, put them in a Python dictionary, and print them out.

(0.5 mark)

In [46]:
from nltk.corpus import brown
sentences = brown.sents()
from gensim.models import Word2Vec
model = Word2Vec(sentences, size=500, min_count=5,iter=50)
gensimDic = {}
for tup in testDic.keys():
    if tup[0] in model.wv and tup[1] in model.wv:
        gensimDic[tup[0],tup[1]] = model.wv.similarity(tup[0],tup[1])
print gensimDic


{('psychology', 'science'): 0.33816621948299463, ('journey', 'car'): 0.2062385560529075, ('brother', 'monk'): 0.04140661896289888, ('equipment', 'maker'): 0.22703857088451185, ('coast', 'forest'): 0.29153825752456985, ('baby', 'mother'): 0.22227691859759607, ('canyon', 'landscape'): 0.18460762791234298, ('doctor', 'personnel'): -0.04960550657035876, ('soap', 'opera'): 0.008254358159182719, ('monk', 'slave'): 0.009712031620398746, ('psychology', 'doctor'): -0.008227266997057647, ('car', 'automobile'): 0.16282280044958908, ('type', 'kind'): 0.2582891534028271, ('psychology', 'health'): 0.14902048179491295, ('computer', 'laboratory'): 0.1873652730652104, ('century', 'year'): 0.30776580471076775, ('coast', 'hill'): 0.4316570672686951, ('psychology', 'mind'): 0.06316847958871596, ('luxury', 'car'): 0.10423690252178615, ('school', 'center'): -0.024055710325684473, ('journey', 'voyage'): 0.4872257372086407, ('stock', 'egg'): 0.16348277787083756, ('word', 'similarity'): 0.03454630124083606, ('


**Instructions:** Finally, you should compare all the similarities you've created to the gold standard you loaded and filtered in the first step. For this, you can use the Pearson correlation co-efficient (`pearsonr`), which is included in scipy (`scipy.stats`). Be careful converting your dictionaries to lists for this purpose, the data for the two datasets needs to be in the same order for correct comparison using correlation. Write a general function, then apply it to each of the similarity score dictionaries, and print out the result for each (be sure to label them!). Hint: All of the methods used here should be markedly above 0, but also far from 1 (perfect correlation); if you're not getting reasonable results, go back and check your code for bugs!  

(0.5 mark)


In [47]:
from scipy.stats import pearsonr
keyList = testDic.keys()
testDicList = [testDic[tup] for tup in keyList]
PMIMapList = [PMIMap[tup] for tup in keyList]
cosinMList = [cosinM[tup] for tup in keyList]
wupDicList = [wupDic[tup] for tup in keyList]
gensimDicList = [gensimDic[tup] for tup in keyList]

print "Wu-Palmer: "+str(pearsonr(testDicList, wupDicList))
print "PPMI: "+str(pearsonr(testDicList, PMIMapList))
print "SVD: "+str(pearsonr(testDicList, cosinMList))
print "gensim: "+str(pearsonr(testDicList, gensimDicList))


Wu-Palmer: (0.4566927406366401, 0.016636665403273413)
PPMI: (0.22129590801974985, 0.2673008317795401)
SVD: (0.3421048121494456, 0.08069704554003701)
gensim: (0.3172843594172951, 0.10682525887926714)


## Bonus: Improving the correlation

**Instructions:** To get the extra credit, you should try to get a noticeably (at least 0.05) higher correlation than that of the highest correlation that you found in the main assignment. To accomplish this, you can use any resources included or accessible via NLTK EXCEPT the Google word2vec vectors. Half of your mark here will be based on the thought and effort involved in your solution (if you tried something sensible that didn't work, show us), the other half will be based on the actual performance increase, which you should demonstrate. For full credit, be sure to explain exactly what you are doing.

(1 bonus mark) 

In [50]:
import nltk
nltk.download('wordnet_ic')
from nltk.corpus import wordnet_ic
brown_ic = wordnet_ic.ic('ic-brown.dat')
linDic = {}
for tup in s:
    linDic[tup] = testDicSyn[tup[0]].lin_similarity(testDicSyn[tup[1]],brown_ic)
# print lchDic


from scipy.stats import pearsonr
testDicList = [testDic[tup] for tup in keyList]
wupDicList = [wupDic[tup] for tup in keyList]
linDicList = [linDic[tup] for tup in keyList]

print "Wu-Palmer: "+str(pearsonr(testDicList, wupDicList))
print "lin: "+str(pearsonr(testDicList, linDicList))
print "impiroved correlation: " + str(pearsonr(testDicList, linDicList)[0] - pearsonr(testDicList, wupDicList)[0])
print

print "Lin Similarity: Return a score denoting how similar two word senses are"
print ", based on the Information Content (IC) of the Least Common Subsumer"
print "(most specific ancestor node) and that of the two input Synsets (and I use brown_ic here)"
print "and I try some other method such as Resnik Similarity,Jiang-Conrath Similarity,Lin Similarity"
print "Lin Similarity got the highest correlation among them."
print "The relationship is given by the equation 2 * IC(lcs) / (IC(s1) + IC(s2))."

[nltk_data] Downloading package wordnet_ic to /Users/mao/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!
Wu-Palmer: (0.4566927406366401, 0.016636665403273413)
lin: (0.5235797314329922, 0.005066605076925647)
impiroved correlation: 0.06688699079635207

Lin Similarity: Return a score denoting how similar two word senses are
, based on the Information Content (IC) of the Least Common Subsumer
(most specific ancestor node) and that of the two input Synsets (and I use brown_ic here)
and I try some other method such as Resnik Similarity,Jiang-Conrath Similarity,Lin Similarity
Lin Similarity got the highest correlation among them.
The relationship is given by the equation 2 * IC(lcs) / (IC(s1) + IC(s2)).


## A final word

Normally, we would not use a corpus as small as the Brown for the purposes of building distributional word vectors. Also, note that filtering our test set to just words we are likely to do well on would typically be considered cheating.

asdfasdf
